In [196]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
import pandas as pd
import soundfile as sf
import numpy as np
from sklearn.utils import shuffle
from sys import platform
from sklearn.model_selection import train_test_split, GroupShuffleSplit

if platform == "linux" or platform == "linux2":
    # linux
    path='/home/vkalbag/projects/'
elif platform == "darwin":
    # OS X
    path='/Users/vedant/Desktop/Programming/'

In [197]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']

d=np.load(f'{path}ScreamDetection/resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'{path}/ScreamDetection/resources/dataset/lookup_new.csv')

In [198]:
df=df.merge(lut[['video_id','band_name']],on='video_id')
df

,video_id,start_time,mid_ts,label,audio,vggish,band_name
0,4600fGWcn9o,0.0,0.5,no_vocals,"[0.0, 0.0, 0.0, -3.0517578125e-05, -1.52587890...","[166.0, 8.0, 149.0, 128.0, 199.0, 57.0, 96.0, ...",Textures
1,4600fGWcn9o,0.5,1.0,no_vocals,"[0.0004730224609375, 0.0001983642578125, -6.10...","[175.0, 10.0, 147.0, 103.0, 210.0, 74.0, 81.0,...",Textures
2,4600fGWcn9o,1.0,1.5,no_vocals,"[-4.57763671875e-05, -3.0517578125e-05, 1.5258...","[173.0, 10.0, 148.0, 131.0, 191.0, 76.0, 71.0,...",Textures
3,4600fGWcn9o,1.5,2.0,no_vocals,"[3.0517578125e-05, 0.0, -3.0517578125e-05, 0.0...","[164.0, 7.0, 154.0, 128.0, 190.0, 80.0, 29.0, ...",Textures
4,4600fGWcn9o,2.0,2.5,no_vocals,"[-0.0008087158203125, -0.000885009765625, -0.0...","[169.0, 11.0, 146.0, 115.0, 191.0, 75.0, 108.0...",Textures
...,...,...,...,...,...,...,...
33815,0m5fIHHfJTw,217.5,218.0,no_vocals,"[1.52587890625e-05, 6.103515625e-05, 0.0001220...","[174.0, 13.0, 142.0, 130.0, 194.0, 76.0, 84.0,...",Lamb of God
33816,0m5fIHHfJTw,218.0,218.5,no_vocals,"[6.103515625e-05, 9.1552734375e-05, 0.00012207...","[174.0, 10.0, 148.0, 124.0, 190.0, 77.0, 86.0,...",Lamb of God
33817,0m5fIHHfJTw,218.5,219.0,no_vocals,"[0.0, -1.52587890625e-05, -1.52587890625e-05, ...","[171.0, 9.0, 140.0, 110.0, 201.0, 56.0, 68.0, ...",Lamb of God
33818,0m5fIHHfJTw,219.0,219.5,no_vocals,"[-1.52587890625e-05, -1.52587890625e-05, -1.52...","[172.0, 10.0, 141.0, 109.0, 203.0, 56.0, 71.0,...",Lamb of God


In [199]:
feature_df=df[['label','audio','band_name']]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(3,'label_mapped',mapping)

## Undersampling the master data

In [202]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy={0:2462,1:3000,2:3000},random_state=0)
X = feature_df[['audio','band_name']].to_numpy()
y=feature_df['label_mapped'].to_numpy()
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

In [203]:
d=pd.DataFrame()
d['y_under'] = y_under
d['blah'] = 1

print(d.groupby('y_under')['blah'].count())

y_under
0    2462
1    3000
2    3000
Name: blah, dtype: int64


## Creating train-test-validation split

In [204]:
gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=42)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

In [205]:
X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=42)

In [207]:
# d=pd.DataFrame(X_train)
# d['blah'] = 1
# print('TRAIN')
# print(d.groupby(1).count())


# d=pd.DataFrame(X_test)
# d['blah'] = 1
# print('TEST')
# print(d.groupby(1).count())


In [208]:
d=pd.DataFrame()
d['y_train'] = y_train
d['blah'] = 1
print('TRAIN')
print(d.groupby('y_train')['blah'].count())
train = d['blah'].sum()

d=pd.DataFrame()
d['y_test'] = y_test
d['blah'] = 1
print('TEST')
print(d.groupby('y_test')['blah'].count())
test = d['blah'].sum()
d=pd.DataFrame()
d['y_valid'] = y_valid
d['blah'] = 1
print('VALID')
print(d.groupby('y_valid')['blah'].count())
valid = d['blah'].sum()

print(f"Train:Test:Validation - {train}:{test}:{valid}")

TRAIN
y_train
0    1595
1    2057
2    2156
Name: blah, dtype: int64
TEST
y_test
0    430
1    470
2    427
Name: blah, dtype: int64
VALID
y_valid
0    437
1    473
2    417
Name: blah, dtype: int64
Train:Test:Validation - 5808:1327:1327


In [211]:
d=pd.DataFrame()
d['y_train'] = y_train
d['blah'] = 1
print('TRAIN')
print(d.groupby('y_train')['blah'].count())
train = d['blah'].sum()

d=pd.DataFrame()
d['y_test'] = y_test
d['blah'] = 1
print('TEST')
print(d.groupby('y_test')['blah'].count())
test = d['blah'].sum()
d=pd.DataFrame()
d['y_valid'] = y_valid
d['blah'] = 1
print('VALID')
print(d.groupby('y_valid')['blah'].count())
valid = d['blah'].sum()

print(f"Train:Test:Validation - {train}:{test}:{valid}")

TRAIN
y_train
0    1595
1    2057
2    2156
Name: blah, dtype: int64
TEST
y_test
0    430
1    470
2    427
Name: blah, dtype: int64
VALID
y_valid
0    437
1    473
2    417
Name: blah, dtype: int64
Train:Test:Validation - 5808:1327:1327


In [212]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot)
X_test,y_test=shuffle(X_test,y_test_hot)
X_valid,y_valid=shuffle(X_valid,y_valid_hot)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_train-rawaudio.npy', X_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_test-rawaudio.npy', X_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_valid-rawaudio.npy', X_valid)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_train-rawaudio.npy', y_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_test-rawaudio.npy', y_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_valid-rawaudio.npy', y_valid)

## VGGish

In [214]:
feature_df=df[['label','vggish','band_name']]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(3,'label_mapped',mapping)


from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy={0:2462,1:3000,2:3000},random_state=0)
X = feature_df[['vggish','band_name']].to_numpy()
y=feature_df['label_mapped'].to_numpy()
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=42)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=42)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot)
X_test,y_test=shuffle(X_test,y_test_hot)
X_valid,y_valid=shuffle(X_valid,y_valid_hot)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_train-vggish.npy', X_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_test-vggish.npy', X_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_valid-vggish.npy', X_valid)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_train-vggish.npy', y_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_test-vggish.npy', y_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_valid-vggish.npy', y_valid)

## Features

In [ ]:
cols=['video_id','start_time','mid_ts','label','audio','vggish']

d=np.load(f'{path}ScreamDetection/resources/working_data/vocal_only_data_with_vggish.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(f'{path}/ScreamDetection/resources/dataset/lookup_new.csv')


feature_df=df[['label','vggish','band_name']]
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(3,'label_mapped',mapping)


from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy={0:2462,1:3000,2:3000},random_state=0)
X = feature_df[['vggish','band_name']].to_numpy()
y=feature_df['label_mapped'].to_numpy()
X_under, y_under = undersample.fit_resample(X, y)

band_names = X_under[:,1]
X_under=X_under[:,0]#.reshape(-1,1).flatten()
y_under=y_under

gss = GroupShuffleSplit(n_splits=5, train_size=.7, random_state=42)
train,test = next(gss.split(X_under, y_under, band_names))
# for train_idx,test_idx in gss.split(X_under, y_under, band_names): 
#     print(train_idx,test_idx)

X_train = X_under[train]
X_test1 = X_under[test]

y_train = y_under[train]
y_test1 = y_under[test]

X_test,X_valid,y_test,y_valid = train_test_split(X_test1, y_test1,test_size=0.5,random_state=42)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_valid_hot = to_categorical(y_valid)

X_train,y_train=shuffle(X_train,y_train_hot)
X_test,y_test=shuffle(X_test,y_test_hot)
X_valid,y_valid=shuffle(X_valid,y_valid_hot)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_train-vggish.npy', X_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_test-vggish.npy', X_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/x_valid-vggish.npy', X_valid)

np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_train-vggish.npy', y_train)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_test-vggish.npy', y_test)
np.save(f'{path}ScreamDetection/FINAL/working_data_final/y_valid-vggish.npy', y_valid)